In [2]:
import pandas as pd

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Github/Patricionog/Modulio')

import Foragio as fg
import Excelio as ex
import Framio as fr
import Stringio as st
import Listio as ls
import Graphio as gr
import Numbio as nm

# Imprimir el docstring.
# print(Function.__doc__)

In [3]:
# Diccionario de lenguaje-nombre carpeta de Google Drive.
Drive_Names_By_Language = {'Español': 'Mi unidad',
               'Inglés': 'My Drive'}

In [4]:
# El lenguaje de Windows en la PC.
PC_Language = 'Inglés'

### __Aumentos porcentuales__
***

In [5]:
Aumentos = {'9 Lives' : 20}

### __Database de marcas y subcategorías__
***

In [6]:
# Pathway del archivo Marcas y subcategorías.
Path_Marcas = f'J:/{Drive_Names_By_Language[PC_Language]}/Forraje/Marcas y subcategorías.xlsx'

In [7]:
# Crear un DataFrame del archivo Marcas y subcategorías.
df_Marcas = pd.read_excel(Path_Marcas)

In [8]:
# Procesamos el DataFrame para evitar errores.
df_Marcas = fr.Process_DataFrame(df_Marcas, Fill_NaN=0)

In [9]:
# Agregamos la columna del Markup.
df_Marcas['Markup'] = df_Marcas['Categoria'].map(lambda x: fg.Markup_And_Unity[x]['Markup %'])

In [10]:
# Chequeamos los nombres de las columnas.
df_Marcas.columns

Index(['Descripcion', 'Subcategoria', 'Marca', 'Categoria', 'Unidad',
       'Markup'],
      dtype='object')

### __Database de Nextar__
***

In [11]:
# Pathway del archivo Exportar.
Path_Nex = f'J:/{Drive_Names_By_Language[PC_Language]}/Forraje/Exportar.xls'

In [12]:
# Crear un DataFrame del archivo Exportar.
df_Nex = pd.read_excel(Path_Nex)

In [13]:
# Mantenemos algunas columnas.
df_Nex_Columnas = ['Descripción', 'Costo Unitário']

In [14]:
# Procesamos df de Nex para evitar errores.
df_Nex = fr.Process_DataFrame(df_Nex, Columns= df_Nex_Columnas, Remove_Accents_In_Name_Columns=True, 
                              Fill_NaN=0, Replace_In_Name_Columns = [['Costo Unitario'],['Costo']])

In [15]:
# Chequeamos los nombres de las columnas.
df_Nex.columns

Index(['Descripcion', 'Costo'], dtype='object')

### __Database agrupado__
***

In [16]:
# Creamos otro basándonos en el de Nex.
df = df_Marcas.copy()

In [17]:
# Comprobamos que las filas de los dos dfs no difieren.
df_Differences = fr.Compare_Columns(df_Marcas, 'Descripcion', df_Nex, 'Descripcion',
                   'Marcas', 'Nex')

if len(df_Differences) == 0:
    print("Todo marcha bien, muchacho.")
else:
    print(df_Differences)

Todo marcha bien, muchacho.


In [18]:
# Agregar columna de Costos.
df['Costo'] = df_Nex['Costo'].astype(int)

In [19]:
# Convertir Markup a formato int.
df['Markup'] = df['Markup'].astype(int)

In [20]:
# Agregar columna de Precio.
df['Precio'] = df['Costo'] * (1 + (df['Markup']/100))

df.columns

Index(['Descripcion', 'Subcategoria', 'Marca', 'Categoria', 'Unidad', 'Markup',
       'Costo', 'Precio'],
      dtype='object')

In [21]:
for Marca, Aumento_Porcentual in Aumentos.items():
    df = fr.Apply_Operations_To_Selected_Rows(
        df, 
        Filtered_Column = 'Marca', 
        Filter_Value = Marca, 
        Condition = 'Match', 
        Columns_To_Operate = ['Precio'],
        Operations = [lambda x: x * (1 + (Aumento_Porcentual/100))])

In [22]:
# Redondear columna de Precio.
df['Precio'] = df['Precio'].apply(lambda x: nm.Round_By_Multiple(x, 100, Rounding_Direction='Upper'))

### __Database con aumentos__
***

In [23]:
# Filtrar df solo con las filas aumentadas.
df_Aumentos = df[df['Marca'].isin(Aumentos.keys())]

In [24]:
df_Aumentos

,Descripcion,Subcategoria,Marca,Categoria,Unidad,Markup,Costo,Precio
0,9 Lives Gatitos,Grupo Pilar,9 Lives,Gato Granel,KG,48,2310,4100
1,9 Lives Gatitos x 8 kg.,Grupo Pilar,9 Lives,Gato,UN,32,18510,29400
2,9 Lives Pescado,Grupo Pilar,9 Lives,Gato Granel,KG,48,2000,3700
3,9 Lives Pescado x 15 kg.,Grupo Pilar,9 Lives,Gato,UN,32,29970,47500


In [29]:
import pyautogui
import pyperclip
import time
import keyboard
import subprocess

def Abort_Program():

    print("Se presionó la tecla de aborto. El programa está haciendo cagadas.")
    exit()

# Configura la tecla de aborto.
keyboard.add_hotkey('ctrl+shift+a', Abort_Program)

def Open_Window(Window_Title, Time_Sleep = 0):

    time.sleep(Time_Sleep) 
    Windows = pyautogui.getWindowsWithTitle(Window_Title)

    if Windows:
        Windows[0].activate()
        time.sleep(Time_Sleep) 

        # Hacer clic en el centro de la ventana.
        pyautogui.click(Windows[0].left + Windows[0].width // 2, Windows[0].top + Windows[0].height // 2)
        time.sleep(Time_Sleep) 
    else:
        print(f"Window '{Window_Title}' not found.")
        return

def Open_Program(Program_Path, Time_Sleep = 5):
    subprocess.Popen(Program_Path)
    time.sleep(Time_Sleep)

def Transcribir_Aumentos(df_Aumentos, 
                         Product_Column, 
                         Cost_Column,
                         Price_Column,
                         Time_Sleep = 0.5):

    
    # Abre el programa de NexAdmin.
    Program_Path = "C:/Nex/NexAdmin.exe"  
    Open_Program(Program_Path)

    # Abrir pestaña de Nex.
    Open_Window("NexAdmin", Time_Sleep)

    for Index, Product in enumerate(df[Product_Column]):

        # Copiar descripción al portapapeles.
        pyperclip.copy(Product)

        # Hacer clic en una posición de Nex para activar la pantalla.
        pyautogui.click(x=577, y=371)
        time.sleep(Time_Sleep)

        # Navegar por opciones hasta casilla Producto.
        pyautogui.press('tab', presses=4)
        time.sleep(Time_Sleep)

        # Pegar Producto.
        pyautogui.hotkey('ctrl', 'v')
        time.sleep(Time_Sleep*3)
        pyautogui.click(x=486, y=314)
        time.sleep(Time_Sleep)
        pyautogui.press('enter')
        time.sleep(Time_Sleep)

        # Copiar Precio.
        pyperclip.copy(df.loc[Index, Price_Column])

        # Navegar por opciones hasta casilla de Precio.
        pyautogui.press('tab', presses=8)
        time.sleep(Time_Sleep)
        pyautogui.press('space')
        time.sleep(Time_Sleep)

        # Borrar el contenido anterior de Precio.
        pyautogui.press('del')
        time.sleep(Time_Sleep)

        # Escribir el Precio actualizado.
        pyautogui.hotkey('ctrl', 'v')
        time.sleep(Time_Sleep)

        # Copiar Costo.
        pyperclip.copy(df.loc[Index, Cost_Column])

        # Navegar por opciones hasta casilla de Costo.
        pyautogui.press('tab', presses=3)
        time.sleep(Time_Sleep)

        # Borrar el contenido anterior de Costo.
        pyautogui.press('del')
        time.sleep(Time_Sleep)

        # Escribir el Costo actualizado.
        pyautogui.hotkey('ctrl', 'v')
        time.sleep(Time_Sleep)

        # Presionar F2 y la flecha hacia abajo
        pyautogui.press('f2')
        time.sleep(Time_Sleep)
        pyautogui.press('space')
        time.sleep(Time_Sleep)
        pyautogui.click(x=486, y=314)
        time.sleep(Time_Sleep)

Transcribir_Aumentos(df_Aumentos, 'Descripcion', 'Costo', 'Precio')
                                 

: 

In [55]:
from pywinauto import Application

# Conectar a la aplicación Nex.
App = Application(backend="win32").connect(process=11140)

C:\Users\tomas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


In [48]:
# Acceder al primer panel (ajusta el título o la clase según sea necesario)
panel_1 = Window.child_window(class_name="TPanel")
panel_1.wait('visible')  # Esperar a que el panel sea visible
panel_1.set_focus()  # Enfocar el panel


TimeoutError: timed out

In [ ]:

# Acceder al segundo panel
panel_2 = panel_1.child_window(title="Panel 2 Title", class_name="Panel2Class")
panel_2.wait('visible')
panel_2.set_focus()

# Finalmente, acceder al botón dentro del último panel
button = panel_2.child_window(class_name="TnexButtonMenuPri10")

# Verificar si el botón está visible y habilitado antes de hacer clic
if button.is_visible() and button.is_enabled():
    button.click()
else:
    print("El botón no está visible o habilitado.")